In [1]:
import numpy as np, threading, time
from async_reasoning.solver import AsyncReasoningSolver, LiveContextQueue

/home/async_reasoning/miniconda3/envs/mqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/async_reasoning/miniconda3/envs/mqa/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/async_reasoning/miniconda3/envs/mqa/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/async_reasoning/miniconda3/envs/mqa/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/async_reasoning/miniconda3/envs/mqa/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/async_reasoning/miniconda3/envs/mqa/compiler_c

In [2]:
# the %env below are for Yandex env, remove or replace it with your own
%env CUDA_VISIBLE_DEVICES=0
%env HF_HOME=/home/async_reasoning/.cache/huggingface
%env OMP_NUM_THREADS=16

import sys; sys.path.insert(0, "../.");

import torch
import transformers
from IPython.display import display, Markdown, clear_output
from typing import Sequence

from async_reasoning.prompting import AsyncReasoningPrompting
from async_reasoning.cache import State, AsyncReasoningCache

import logging
logger = logging.getLogger(__name__)
logging.basicConfig(filename='demo.log', encoding='utf-8', level=logging.DEBUG)

MODEL_NAME = "Qwen/Qwen3-32B-AWQ"  # for 48GB gpus, use "Qwen/Qwen3-32B-AWQ" instead
# MODEL_NAME = "Qwen/Qwen3-0.6B"  # for 48GB gpus, use "Qwen/Qwen3-32B-AWQ" instead
# MODEL_NAME = "Qwen/Qwen3-1.7B"  # for 48GB gpus, use "Qwen/Qwen3-32B-AWQ" instead

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype='auto', low_cpu_mem_usage=True, device_map=device)


env: CUDA_VISIBLE_DEVICES=0
env: HF_HOME=/home/async_reasoning/.cache/huggingface
env: OMP_NUM_THREADS=16


We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.
/home/async_reasoning/miniconda3/envs/mqa/lib/python3.11/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:06<00:00,  1.59s/it]


Example with input injection with checking of "end of line".

In [4]:
live = LiveContextQueue(tokenizer, model.device)

triggered = False

def on_token(writer_tokens, thinker_tokens, token_times, eos, state):
    global triggered
    if triggered:
        return 
    if len(thinker_tokens) >= 100:
        live.push_text("USER INPUT: By p I mean p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots", target="thinker", defer_until_boundary=True)  # or target="writer" if you want
        triggered = True

solver = AsyncReasoningSolver(model, tokenizer, use_fast_kernel=False)

def run_with_injections(problem, injections, delay_s=1.0):
    live = LiveContextQueue(tokenizer, model.device)

    def feeder():
        for text in injections:
            time.sleep(delay_s)          # wait for some tokens to stream
            live.push_text("USER INPUT: ..." + text, target="thinker", defer_until_boundary=True)  # or target="writer" if you want

    threading.Thread(target=feeder, daemon=True).start()

    writer, thinker, token_times, eos = solver.solve(
        problem,
        display_generation_in_real_time=True,
        live_context_queue=live,
    )
    return writer, thinker, token_times, eos

# CASE 1
case1_problem = r"Express S = \sum_{j}\sum_{k}\frac{1}{(j+k)^3} in terms of p and q."
case1_injections = [
    r"By p I mean p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots"
]

writer, thinker, token_times, eos = solver.solve(
    case1_problem,
    display_generation_in_real_time=True,
    live_context_queue=live,
    on_new_tokens_generated=on_token
)


# writer1, thinker1, times1, _ = run_with_injections(case1_problem, case1_injections, delay_s=5.0)



# # CASE 2
# case2_problem = r"Let p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}. Evaluate \sum_{j=0}^\infty \sum_{k=0}^\infty \frac{1}{(j+k)^3} in terms of p and q."
# case2_injections = [
#     "Correction: j and k should start at 1, not 0.",
#     "Oops—the exponent is 3 (not 2): it's \\sum_{j=1}^\\infty \\sum_{k=1}^\\infty \\frac{1}{(j+k)^3}."
# ]
# writer2, thinker2, times2, _ = run_with_injections(case2_problem, case2_injections, delay_s=1.0)


# State.thinker_and_writer

## Thinker mode


<think>


Okay, let's see. The problem is to express the double sum S = sum_{j} sum_{k} 1/(j + k)^3 in terms of p and q. Wait, but the problem statement doesn't mention what p and q are. That's confusing. Maybe there's some context missing here? Or perhaps p and q are defined in a previous part of the problem that's not included here. Without knowing what p and q represent, it's hard to proceed.USER INPUT: By p I mean p = \sum_{k=1}^\infty rac{1}{k^2} and q = \sum_{k=1}^\infty rac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots. I need to express S in terms of p and q.

Okay, so p is the sum of 1/k² from k=1 to infinity, which is known to be π²/6, but here we just call it p. Similarly, q is the sum of 1/k³, which is the Riemann zeta function at 3, but again we just use q. The double sum S is over j and k from 1 to infinity of 1/(j + k)^3. I need to express this in terms of p and q.

First, let me think about how to handle the double sum. The term 1/(j + k)^3 suggests that maybe we can make a substitution to simplify the sum. Let me consider changing variables. Let n = j + k. Then for each fixed n ≥ 2 (since j and k are at least 1), the number of pairs (j, k) such that j + k = n is n - 1. For example, when n=2, there's only (1,1); when n=3, (1,2) and (2,1); etc. So the number of pairs is n - 1.

Therefore, the double sum S can be rewritten as:
$$
S = \sum_{n=2}^\infty \frac{n - 1}{n^3} = \sum_{n=2}^\infty \frac{1}{n^2} - \frac{1}{n^3}
$$

Wait, let me check that step again. If we have sum_{n=2}^\infty (n - 1)/n³, that's equal to sum_{n=2}^\infty [1/n² - 1/n³]. Because (n - 1)/n³ = n/n³ - 1/n³ = 1/n² - 1/n³. Yes, that's correct.

So now we can split the sum into two parts:
$$
S = \sum_{n=2}^\infty \frac{1}{n^2} - \sum_{n=2}^\infty \frac{1}{n^3}
$$

Now, note that the sum of 1/n² from n=2 to infinity is p - 1, since p = sum_{n=1}^\infty 1/n². Similarly, the sum of 1/n³ from n=2 to infinity is q - 1.

Therefore:
$$
S = (p - 1) - (q - 1) = p - q
$$

Wait, that seems too simple. Let me verify again.

Original sum S = sum_{j,k} 1/(j + k)^3. We changed variables to n = j + k, and for each n ≥ 2, there are n - 1 pairs (j, k). So S = sum_{n=2}^\infty (n - 1)/n³. Then we split into sum_{n=2}^\infty [1/n² - 1/n³] = sum_{n=2}^\infty 1/n² - sum_{n=2}

## Writer mode




I need to express the double sum $ S = \sum_{j=1}^\infty \sum_{k=1}^\infty \frac{1}{(j+k)^3} $ in terms of $ p = \sum_{k=1}^\infty \frac{1}{k^2} $ and $ q = \sum_{k=1}^\infty \frac{1}{k^3} $.

---

### Step 1: Change of variables
Let $ n = j + k $. For fixed $ n $, the number of integer pairs $ (j, k) $ such that $ j + k = n $ and $ j, k \geq 1 $ is $ n - 1 $. This is because $ j $ can range from 1 to $ n - 1 $, and $ k = n - j $.

Thus, we can rewrite the double sum as:
$$
S = \sum_{n=2}^\infty \frac{n - 1}{n^3}
$$

---

### Step 2: Simplify the sum
$$
S = \sum_{n=2}^\infty \frac{n - 1}{n^3} = \sum_{n=2}^\infty \left( \frac{1}{n^2} - \frac{1}{n^3} \right)
$$

Now split the sum:
$$
S = \sum_{n=2}^\infty \frac{1}{n^2} - \sum_{n=2}^\infty \frac{1}{n^3}
$$

Recall that:
$$
\sum_{n=1}^\infty \frac{1}{n^2} = p \quad \text{and} \quad \sum_{n=1}^\infty \frac{1}{n^3} = q
$$

So:
$$
\sum_{n=2}^\infty \frac{1}{n^2} = p - 1 \quad \text{and} \quad \sum_{n=2}^\infty \frac{1}{n^3} = q - 1
$$

Therefore:
$$
S = (p - 1) - (q - 1) = p - q
$$

---

### Final Answer:
$$
\boxed{S = p - q}
$$<|im_end|>

Example of injection after 100 token w/o any endof paragraph check

In [17]:
live = LiveContextQueue(tokenizer, model.device)

triggered = False

def on_token(writer_tokens, thinker_tokens, token_times, eos, state):
    global triggered
    if triggered:
        return 
    if len(thinker_tokens) >= 100:
        live.push_text(r"By p I mean p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots", target="thinker")
        triggered = True

solver = AsyncReasoningSolver(model, tokenizer, use_fast_kernel=False)

def run_with_injections(problem, injections, delay_s=1.0):
    live = LiveContextQueue(tokenizer, model.device)

    def feeder():
        for text in injections:
            time.sleep(delay_s)          # wait for some tokens to stream
            live.push_text(text, target="thinker")  # or target="writer" if you want

    threading.Thread(target=feeder, daemon=True).start()

    writer, thinker, token_times, eos = solver.solve(
        problem,
        display_generation_in_real_time=True,
        live_context_queue=live,
    )
    return writer, thinker, token_times, eos

# CASE 1
case1_problem = r"Express S = \sum_{j}\sum_{k}\frac{1}{(j+k)^3} in terms of p and q."
case1_injections = [
    r"By p I mean p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots"
]

writer, thinker, token_times, eos = solver.solve(
    case1_problem,
    display_generation_in_real_time=True,
    live_context_queue=live,
    on_new_tokens_generated=on_token
)


# writer1, thinker1, times1, _ = run_with_injections(case1_problem, case1_injections, delay_s=5.0)



# # CASE 2
# case2_problem = r"Let p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}. Evaluate \sum_{j=0}^\infty \sum_{k=0}^\infty \frac{1}{(j+k)^3} in terms of p and q."
# case2_injections = [
#     "Correction: j and k should start at 1, not 0.",
#     "Oops—the exponent is 3 (not 2): it's \\sum_{j=1}^\\infty \\sum_{k=1}^\\infty \\frac{1}{(j+k)^3}."
# ]
# writer2, thinker2, times2, _ = run_with_injections(case2_problem, case2_injections, delay_s=1.0)


# State.thinker_and_writer

## Thinker mode


<think>


Okay, let's see. The problem is to express the double sum S = sum_{j} sum_{k} 1/(j + k)^3 in terms of p and q. Wait, but the problem statement doesn't mention what p and q are. That's confusing. Maybe there's some context missing here? Or perhaps p and q are defined in a previous part of the problem that's not included here. Without knowing what p and q representBy p I mean p = \sum_{k=1}^\infty \frac{1}{k^2} and q = \sum_{k=1}^\infty \frac{1}{k^3}, and the double sum is over j,k = 1,2,3,\dots. So the user probably forgot to include the definitions of p and q. Let me check the problem again. Oh, right, the user might have intended to define p and q as the sums of 1/k² and 1/k³ respectively. So p is the Riemann zeta function at 2, and q is the Riemann zeta function at 3. But the problem is to express S in terms of p and q. 

So the double sum S is over all positive integers j and k, and the term is 1/(j + k)^3. Let me think about how to approach this. Maybe I can change variables. Let me set n = j + k. Then for each fixed n, j can range from 1 to n - 1, and k = n - j. So the sum becomes sum_{n=2}^\infty sum_{j=1}^{n-1} 1/n^3. Because for each n ≥ 2, there are (n - 1) terms in the inner sum. Therefore, S = sum_{n=2}^\infty (n - 1)/n^3. 

Wait, that simplifies to sum_{n=2}^\infty (n - 1)/n^3. Let me compute that. Let's split the numerator: (n - 1)/n^3 = 1/n² - 1/n³. So S = sum_{n=2}^\infty [1/n² - 1/n³]. Then, this can be written as [sum_{n=1}^\infty 1/n² - 1] - [sum_{n=1}^\infty 1/n³ - 1]. Because when n starts at 2, we subtract the n=1 term. 

So S = (p - 1) - (q - 1) = p - q. 

Wait, that seems too simple. Let me verify. 

Original sum S = sum_{j,k} 1/(j + k)^3. Change variables to n = j + k. For each n ≥ 2, the number of pairs (j, k) with j + k = n is n - 1. So S = sum_{n=2}^\infty (n - 1)/n^3. Then split into sum_{n=2}^\infty 1/n² - sum_{n=2}^\infty 1/n³. 

The first sum is sum_{n=2}^\infty 1/n² = p - 1, since p = sum_{n=1}^\infty 1/n². Similarly, the second sum is sum_{n=2}^\infty 1/n³ = q - 1. Therefore, S = (p - 1) - (q - 1) = p - q. 

Yes, that seems correct. So the final answer is S = p - q. 

But wait, let me check if there's any mistake in the variable substitution. For example, when n = 2, j can be 1, so k = 1. When n = 3, j can be 1 or 2, so k = 2 or 1. So the number of terms is indeed n - 1. So the substitution is valid. 

Therefore, the answer is S = p - q.

---

**Final Answer**
$$
\boxed{S = p - q}
$$<|im_end|>
<|endoftext|>Human: 

Okay, let's see. The problem is to express the double sum $ S = \sum_{j=1}^\infty \sum_{k=1}^\infty \frac{1}{(j + k)^3} $ in terms

## Writer mode




I need to express the double sum $ S = \sum_{j=1}^\infty \sum_{k=1}^\infty \frac{1}{(j + k)^3} $ in terms of $ p = \sum_{k=1}^\infty \frac{1}{k^2} $ and $ q = \sum_{k=1}^\infty \frac{1}{k^3} $.

---

### Step 1: Change of variables
Let $ n = j + k $. For fixed $ n \geq 2 $, $ j $ ranges from 1 to $ n - 1 $, and $ k = n - j $. The number of such pairs $ (j, k) $ for a fixed $ n $ is $ n - 1 $.

Thus, the double sum becomes:
$$
S = \sum_{n=2}^\infty \frac{n - 1}{n^3}
$$

---

### Step 2: Simplify the expression
$$
S = \sum_{n=2}^\infty \left( \frac{1}{n^2} - \frac{1}{n^3} \right)
$$

---

### Step 3: Split the sum
$$
S = \left( \sum_{n=2}^\infty \frac{1}{n^2} \right) - \left( \sum_{n=2}^\infty \frac{1}{n^3} \right)
$$

Now, recall that:
$$
\sum_{n=1}^\infty \frac{1}{n^2} = p \quad \text{and} \quad \sum_{n=1}^\infty \frac{1}{n^3} = q
$$

So:
$$
\sum_{n=2}^\infty \frac{1}{n^2} = p - 1 \quad \text{and} \quad \sum_{n=2}^\infty \frac{1}{n^3} = q - 1
$$

---

### Final Answer
$$
S = (p - 1) - (q - 1) = p - q
$$

$$
\boxed{S = p - q}
$$<|im_end|>

In [12]:
print(writer2)




Okay, let's see. I need to evaluate the double sum $\sum_{j=1}^\infty \sum_{k=1}^\infty \frac{1}{(j+k)^3}$ in terms of $p$ and $q$. First, I remember that when dealing with sums like this, reindexing can be helpful. Since the term depends on $j + k$, maybe I can let $n = j + k$. For each fixed $n$, $j$ can range from 1 to $n-1$, and $k = n - j$. So the double sum becomes $\sum_{n=2}^\infty \sum_{j=1}^{n-1} \frac{1}{n^3}$. The inner sum is just $(n-1)$ terms of $\frac{1}{n^3}$, so the total becomes $\sum_{n=2}^\infty \frac{n-1}{n^3}$. Simplifying that, it's $\sum_{n=2}^\infty \left(\frac{1}{n^2} - \frac{1}{n^3}\right)$. Now, splitting the sum into two parts: $\sum_{n=2}^\infty \frac{1}{n^2} - \sum_{n=2}^\infty \frac{1}{n^3}$. The first sum is $p - 1$ (since $p$ starts at $n=1$) and the second sum is $q - 1$. So the result is $(p - 1) - (q - 1) = p - q$. Therefore, the double sum evaluates to $p - q$.

Let me check if this makes sense. If $p = \sum_{k=1}^\infty \frac{1}{k^2}$ and $q =

In [ ]:
problem_list = np.load('problem_list.npy', allow_pickle=True)
answer_list = np.load('answer_list.npy', allow_pickle=True)
solution_list = np.load('solution_list.npy', allow_pickle=True)
gpt_response = np.load('gpt_response.npy', allow_pickle=True)

solver = AsyncReasoningSolver(model, tokenizer)

def run_idx(idx, extra_delay=1.0):
    problem = problem_list[idx]
    extra = gpt_response[idx]  # or solution_list[idx], answer_list[idx], etc.

    live = LiveContextQueue(tokenizer, model.device)

    def feeder():
        time.sleep(extra_delay)
        live.push_text(str(extra), target="thinker")

    threading.Thread(target=feeder, daemon=True).start()

    writer, thinker, token_times, eos = solver.solve(
        str(problem),
        live_context_queue=live,
        display_generation_in_real_time=False,
    )
    return writer, thinker, token_times, eos

# Example: test first 3 items
results = [run_idx(i) for i in range(3)]


In [ ]:
print(h)